In [274]:
import os

os.getcwd()

'/home/jupyter/work/resources/voxceleb_trainer'

In [40]:
%cd voxceleb_trainer

[Errno 2] No such file or directory: 'voxceleb_trainer'
/home/jupyter/work/resources/voxceleb_trainer


In [275]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [70]:
!python3 ./dataprep.py --save_path /home/jupyter/work/resources/voxceleb_trainer/data --download --user stakhova7 --password dreamjob7  

/home/jupyter/work/ 100%[===================>]  10.00G  18.7MB/s    in 10m 31s 

2022-04-02 18:16:33 (16.2 MB/s) - ‘/home/jupyter/work/resources/voxceleb_trainer/data/vox1_dev_wav_partaa’ saved [10737418240/10737418240]

Checksum successful vox1_dev_wav_partaa.
--2022-04-02 18:19:37--  https://thor.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_dev_wav_partab
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10737418240 (10G) [application/octet-stream]
Saving to: ‘/home/jupyter/work/resources/voxceleb_trainer/data/vox1_dev_wav_partab’

/home/jupyter/work/ 100%[===================>]  10.00G  20.2MB/s    in 9m 55s  

2022-04-02 18:29:33 (17.2 MB/s) - ‘/home/jupyter/work/resources/voxceleb_trainer/data/vox1_dev_wav_partab’ saved [10737418240/10737418240]

Checksum successful vox1_dev_wav_partab.
--2022-04-02 18:32:40--  https:/

In [71]:
!python3 ./dataprep.py --save_path /home/jupyter/work/resources/voxceleb_trainer/data --extract
!python3 ./dataprep.py --save_path /home/jupyter/work/resources/voxceleb_trainer/data --convert

Checksum successful vox1_dev_wav.zip.
Extracting /home/jupyter/work/resources/voxceleb_trainer/data/vox1_dev_wav.zip
mv: cannot stat '/home/jupyter/work/resources/voxceleb_trainer/data/dev/aac/*': No such file or directory
mv: cannot stat '/home/jupyter/work/resources/voxceleb_trainer/data/aac': No such file or directory
Converting files from AAC to WAV
0it [00:00, ?it/s]


In [166]:
#!g1.1
!python3 ./trainSpeakerNet.py --eval --model ResNetSE34V2 --log_input True --encoder_type ASP --n_mels 64 --trainfunc softmaxproto --save_path /home/jupyter/work/resources/voxceleb_trainer/exps/test --eval_frames 400  --initial_model baseline_v2_ap.model


Python Version: 3.8.12 (default, Sep 10 2021, 00:16:05) 
[GCC 7.5.0]
PyTorch Version: 1.9.1+cu111
Number of GPUs: 1
Save path: /home/jupyter/work/resources/voxceleb_trainer/exps/test
Embedding size is 512, encoder ASP.
Initialised Softmax Loss
Initialised AngleProto
Initialised SoftmaxPrototypical Loss
Initialised Adam optimizer
Initialised step LR scheduler
Model baseline_v2_ap.model loaded!
Total parameters:  8028492
Test list data/test_list.txt
Reading 0 of 6: 0.00 Hz, embedding size 512
normalized
Computing 0 of 3: 0.00 Hznormalized
normalized
Traceback (most recent call last):
  File "./trainSpeakerNet.py", line 314, in <module>
    main()
  File "./trainSpeakerNet.py", line 310, in main
    main_worker(0, None, args)
  File "./trainSpeakerNet.py", line 200, in main_worker
    result = tuneThresholdfromScore(sc, lab, [1, 0.1]);
  File "/home/jupyter/work/resources/voxceleb_trainer/tuneThreshold.py", line 25, in tuneThresholdfromScore
    idx = numpy.nanargmin(numpy.absolute((tfa -

Exception: Process exited with code 1

## Генерируем семьи

In [276]:
import torch

embs = torch.load('embeddings.pt')

In [312]:
#делаем словарь вида {'id' : усредненный embedding диктора}

utterance_list = open('./data/train_list.txt', 'r')

mean_embs = dict()
current_id = ''
emb_sum, count = 0, 0

for line in utterance_list:
    id, path = line.split()
    
    if id == current_id:
        emb_sum += embs[path]
        count += 1
    else:
        if current_id:
            mean_embs[current_id] = 1 / count * emb_sum

        emb_sum = embs[path]
        count = 1
        current_id = id
        
    mean_embs[current_id] = 1 / count * emb_sum
        
utterance_list.close()

In [313]:
#!g1.1
#score каждого диктора с каждым
import numpy
import torch.nn.functional as F

scores = dict()
ids = list(mean_embs.keys())

for i in range(len(ids)):
    for j in range(i + 1, len(ids)):
        
        ref_feat = mean_embs[ids[i]].cuda()
        com_feat = mean_embs[ids[j]].cuda()

        dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
        score = numpy.mean(dist);
        scores[(ids[i], ids[j])] = score


In [314]:
similarity_scores = list(scores.values())
min_score = min(similarity_scores)
max_score = max(similarity_scores)
print(min_score, max_score)

-0.19410758 0.8223336


In [315]:
#!g1.1
#делаем матрицу scores

import numpy as np
import torch.nn.functional as F

ids = list(mean_embs.keys())

n = len(ids)
scores = np.zeros((n, n))
id2ind = {ids[i] : i for i in range(n)}

for i in range(n):
    for j in range(i + 1, n):
        
        ref_feat = mean_embs[ids[i]].cuda()
        com_feat = mean_embs[ids[j]].cuda()

        dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
        score = np.mean(dist);
        scores[id2ind[ids[i]], id2ind[ids[j]]] = score


In [316]:
scores = scores + scores.T

In [333]:
threshold = 0.3
ind2id = {i : ids[i] for i in range(n)}

components_mat = scores > threshold

In [345]:
def generate_family(member_id, member_count, components_mat, id2ind, ind2id, relatives_set=set(ids)):
    
    family = {member_id}
    
    while len(family) < len(relatives_set):
        related = np.nonzero(components_mat[id2ind[member_id], :])[0]
        related_ids = [ind2id[i] for i in related]

        relatives_set &= set(related_ids)
        
        if len(family) == member_count:
            return family
        if not relatives_set:
            return family
        
        add_member_id = np.random.choice(list(relatives_set))
        
        #print('current member ', member_id, ' related to ', relatives_set, ' chosen connection to ', add_member_id)
        
        family.add(add_member_id)
        member_id = add_member_id
        
    return family


In [346]:
families = []

for member_count in range(2, 8):
    fixed_size_families = []
    for id_ in ids:
        family = generate_family(id_, member_count, components_mat.astype(int), id2ind, ind2id, relatives_set=set(ids))
        if len(family) == member_count:
            fixed_size_families.append(family)
    families.append(fixed_size_families)

In [343]:
#generate_family('id10104', components_mat.astype(int), id2ind, ind2id, relatives_set=set(ids))

In [351]:
for i, fixed_families in enumerate(families):
    print('nSpeakers', i + 2, len(np.unique(fixed_families)))

nSpeakers 2 1211
nSpeakers 3 1211
nSpeakers 4 1210
nSpeakers 5 1206
nSpeakers 6 1203
nSpeakers 7 1189


## Метрики

In [324]:
def tuneThresholdfromScore(scores, labels, target_fa, target_fr = None):
    
    fpr, tpr, thresholds = metrics.roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr

    tunedThreshold = [];
    if target_fr:
        for tfr in target_fr:
            idx = numpy.nanargmin(numpy.absolute((tfr - fnr)))
            tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]]);
    
    for tfa in target_fa:
        idx = numpy.nanargmin(numpy.absolute((tfa - fpr))) # numpy.where(fpr<=tfa)[0][-1]
        tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]]);
    
    idxE = numpy.nanargmin(numpy.absolute((fnr - fpr)))
    eer  = max(fpr[idxE],fnr[idxE])*100
    
    return (tunedThreshold, eer, fpr, fnr);

# Creates a list of false-negative rates, a list of false-positive rates
# and a list of decision thresholds that give those error-rates.


def ComputeErrorRates(scores, labels):

      # Sort the scores from smallest to largest, and also get the corresponding
      # indexes of the sorted scores.  We will treat the sorted scores as the
      # thresholds at which the the error-rates are evaluated.
      sorted_indexes, thresholds = zip(*sorted(
          [(index, threshold) for index, threshold in enumerate(scores)],
          key=itemgetter(1)))
      sorted_labels = []
      labels = [labels[i] for i in sorted_indexes]
      fnrs = []
      fprs = []

      # At the end of this loop, fnrs[i] is the number of errors made by
      # incorrectly rejecting scores less than thresholds[i]. And, fprs[i]
      # is the total number of times that we have correctly accepted scores
      # greater than thresholds[i].
      for i in range(0, len(labels)):
          if i == 0:
              fnrs.append(labels[i])
              fprs.append(1 - labels[i])
          else:
              fnrs.append(fnrs[i-1] + labels[i])
              fprs.append(fprs[i-1] + 1 - labels[i])
      fnrs_norm = sum(labels)
      fprs_norm = len(labels) - fnrs_norm

      # Now divide by the total number of false negative errors to
      # obtain the false positive rates across all thresholds
      fnrs = [x / float(fnrs_norm) for x in fnrs]

      # Divide by the total number of corret positives to get the
      # true positive rate.  Subtract these quantities from 1 to
      # get the false positive rates.
      fprs = [1 - x / float(fprs_norm) for x in fprs]
      return fnrs, fprs, thresholds


In [411]:
def false_acceptance_rate(labels, labels_true):
    labels, labels_true = np.array(labels), np.array(labels_true)
    return ((labels != 'g') & (labels_true == 'g')).sum() / len(labels) 


def false_rejection_rate(labels, labels_true):
    labels, labels_true = np.array(labels), np.array(labels_true)
    return ((labels == 'g') & (labels_true != 'g')).sum() / len(labels) 


def wrong_member_rate(labels, labels_true):
    labels, labels_true = np.array(labels), np.array(labels_true)
    return ((labels != 'g') & (labels_true != 'g') & (labels != labels_true)).sum() / (labels_true != 'g').sum() 


In [325]:
#сделаем словарь вида {id : [list of path to records]}

f = open('./data/train_list.txt', 'r')
id2records = {ids[i] : [] for i in range(len(ids))}

for line in f:
    id_, path = line.split()
    id2records[id_].append(path)
f.close()

In [503]:
#сделаем пробную генерацию выборки и пропустим ее через tuneThresholdfromScore
#необходимо сделать так, чтобы эмбеддинги из enrolls не пересекались с эмбеддингами из dataset

#member_id - person id 
#count - len(list of utterances from person_id)

def sample_utterance(member_id, count, except_paths=[]):
    path_records = list(np.random.choice(list(set(id2records[member_id]) - set(except_paths)), count))
    sample = [embs[path] for path in path_records]
    return sample, path_records

def generate_enrolls(family_ids, level=4):
    enrolls = dict.fromkeys(family_ids)
    for member_id in family_ids:
        utterances, path_records = sample_utterance(member_id, level)
        enrolls[member_id] = sum(utterances) / level
    return enrolls, path_records
    
from sklearn.utils import shuffle  
def generate_dataset(family_ids, enrolled_paths, ut_per_member=10, ut_per_guest=4, guests_count=4):
    utterances, labels, used_records = [], [], []
    for member_id in family_ids:
        sample, paths = sample_utterance(member_id, ut_per_member, enrolled_paths)
        
        utterances += sample
        used_records += paths
        labels += [member_id] * ut_per_member
        
    guest_ids = np.random.choice(list(set(ids) - set(family_ids)), guests_count)
    for guest_id in guest_ids:
        sample, paths = sample_utterance(guest_id, ut_per_guest, enrolled_paths)
        utterances += sample
        used_records += paths
        labels += ['g'] * ut_per_guest
    
    utterances, labels = shuffle(utterances, labels)
    return utterances, labels, used_records
    

In [479]:
#!g1.1
#берем по 100 примеров каждой семьи
import torch.nn.functional as F

mean_eers = []

for i in range(len(families)):
    eers = []
    for j in range(1000):
        fam = list(families[i][j])
        enrolls, enrolls_path = generate_enrolls(fam)
        utts, labels_true, _ = generate_dataset(fam, enrolls_path)

        scores = []
        labels = []

        for utterance in utts:
            scores_with_enrolled = []
            for enrolled_id, enrolled_ut in enrolls.items():
                ref_feat = utterance.cuda()
                com_feat = enrolled_ut.cuda()
                dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
                score = numpy.mean(dist);
                scores_with_enrolled.append(score)

            scores.append(max(scores_with_enrolled))
            labels.append(list(enrolls.keys())[np.argmax(scores_with_enrolled)])


        far, frr, wmr = [], [], []

        thresholds = sorted(scores)
        labels, scores = np.array(labels), np.array(scores)

        for t in thresholds:
            labels[np.where(scores <= t)[0]] = 'g'
            far.append(false_acceptance_rate(labels, labels_true))
            frr.append(false_rejection_rate(labels, labels_true))
            wmr.append(wrong_member_rate(labels, labels_true))

        fnir, far = np.array(frr) + np.array(wmr), np.array(far)
        idxE = np.nanargmin(np.absolute((far - fnir)))
        eer  = max(far[idxE],fnir[idxE])*100
        eers.append(eer)
    mean_eers.append(np.mean(eers))

In [478]:
#по 100 семей
for i in range(len(mean_eers)):
    print('nSpeakers =', i + 2, 'EER =', mean_eers[i])

nSpeakers = 2 EER = 0.33333333333333326
nSpeakers = 3 EER = 0.48115942028985503
nSpeakers = 4 EER = 0.23928571428571427
nSpeakers = 5 EER = 0.40848484848484845
nSpeakers = 6 EER = 0.8350877192982454
nSpeakers = 7 EER = 0.6714285714285714


In [480]:
#по 1000 семей
for i in range(len(mean_eers)):
    print('nSpeakers =', i + 2, 'EER =', mean_eers[i])

nSpeakers = 2 EER = 0.2716666666666666
nSpeakers = 3 EER = 0.3307246376811594
nSpeakers = 4 EER = 0.41535714285714276
nSpeakers = 5 EER = 0.5258787878787878
nSpeakers = 6 EER = 0.5651754385964912
nSpeakers = 7 EER = 0.5977740863787376


In [508]:
import torch.nn.functional as F

def cosine_score(emb_1, emb_2):
    ref_feat = emb_1.cuda()
    com_feat = emb_2.cuda()
    dist = F.cosine_similarity(ref_feat.unsqueeze(-1), com_feat.unsqueeze(-1).transpose(0,2)).detach().cpu().numpy();
    score = numpy.mean(dist);
    return score

In [ ]:
#!g1.1

simulations = 100
grid_search = []

for a in np.linspace(0.75, 0.95, 3):
    for threshold in np.linspace(0.4, 0.8, 5):

        mean_eers = []
        for i in range(len(families)):
            eers = []
            for j in range(simulations):
                fam = list(families[i][j])

                enrolls, enrolls_path = generate_enrolls(fam)
                train_utts, _, used_in_train = generate_dataset(fam, enrolls_path)
                utts, labels_true, _ = generate_dataset(fam, enrolls_path + used_in_train)

                #training enrolls
                for utterance in train_utts:
                    for enrolled_id, enrolled_ut in enrolls.items():

                        if cosine_score(utterance, enrolled_ut) > threshold:
                            enrolls[enrolled_id] = F.normalize(a * enrolled_ut + (1 - a) * utterance, p=2, dim=1)

                #metrics on test dataset

                scores, labels = [], []

                for utterance in utts:
                    scores_with_enrolled = []

                    for enrolled_id, enrolled_ut in enrolls.items():
                        scores_with_enrolled.append(cosine_score(utterance, enrolled_ut))

                    scores.append(max(scores_with_enrolled))
                    labels.append(list(enrolls.keys())[np.argmax(scores_with_enrolled)])


                far, frr, wmr = [], [], []

                thresholds = sorted(scores)
                labels, scores = np.array(labels), np.array(scores)

                for t in thresholds:
                    labels[np.where(scores <= t)[0]] = 'g'
                    far.append(false_acceptance_rate(labels, labels_true))
                    frr.append(false_rejection_rate(labels, labels_true))
                    wmr.append(wrong_member_rate(labels, labels_true))

                fnir, far = np.array(frr) + np.array(wmr), np.array(far)
                idxE = np.nanargmin(np.absolute((far - fnir)))
                eer  = max(far[idxE],fnir[idxE])*100

                eers.append(eer)

            mean_eers.append(np.mean(eers))

        grid_search.append(mean_eers)
        
        print(a, threshold, grid_search[-1], sep='  |  ')


0.75  |  0.4  |  [2.994444444444444, 13.463768115942027, 14.492857142857142, 21.97151515151515, 27.88157894736842, 32.275415282392025]
0.75  |  0.5  |  [1.4722222222222223, 1.7376811594202897, 1.775, 4.035757575757575, 4.95701754385965, 3.785382059800665]


In [539]:
i = 0
for a in np.linspace(0.75, 0.95, 3):
    for threshold in np.linspace(0.4, 0.8, 5):
        print('a =', a, 'threshold =', threshold)
        print(grid_search[i])
        print()
        i += 1

a = 0.75 threshold = 0.4
[1.4444444444444446, 6.8478260869565215, 6.946428571428572, 13.093333333333332, 19.71228070175438, 23.912292358803985]

a = 0.75 threshold = 0.5
[0.8055555555555556, 1.0869565217391304, 1.4964285714285714, 3.213333333333334, 5.185964912280701, 5.819933554817275]

a = 0.75 threshold = 0.6000000000000001
[0.47222222222222215, 0.5231884057971015, 0.4107142857142857, 1.1866666666666668, 1.1692982456140348, 0.7544850498338872]

a = 0.75 threshold = 0.7000000000000001
[0.33333333333333326, 0.34782608695652173, 0.3928571428571428, 0.509090909090909, 0.6263157894736842, 0.8259136212624584]

a = 0.75 threshold = 0.8
[0.8611111111111112, 0.43623188405797103, 0.3928571428571428, 0.5048484848484849, 0.6763157894736841, 0.764451827242525]

a = 0.85 threshold = 0.4
[1.5888888888888888, 6.230434782608696, 8.239285714285714, 14.48969696969697, 16.923684210526318, 25.39435215946844]

a = 0.85 threshold = 0.5
[0.8388888888888889, 1.2478260869565219, 1.675, 2.7624242424242427, 4.

In [550]:
#!g1.1

a, threshold = 0.85, 0.7
simulations = 1000


mean_eers = []
for i in range(len(families)):
    eers = []
    for j in range(simulations):
        fam = list(families[i][j])

        enrolls, enrolls_path = generate_enrolls(fam)
        train_utts, _, used_in_train = generate_dataset(fam, enrolls_path)
        utts, labels_true, _ = generate_dataset(fam, enrolls_path + used_in_train)

        #training enrolls
        for utterance in train_utts:
            for enrolled_id, enrolled_ut in enrolls.items():

                if cosine_score(utterance, enrolled_ut) > threshold:
                    enrolls[enrolled_id] = F.normalize(a * enrolled_ut + (1 - a) * utterance, p=2, dim=1)

        #metrics on test dataset

        scores, labels = [], []

        for utterance in utts:
            scores_with_enrolled = []

            for enrolled_id, enrolled_ut in enrolls.items():
                scores_with_enrolled.append(cosine_score(utterance, enrolled_ut))

            scores.append(max(scores_with_enrolled))
            labels.append(list(enrolls.keys())[np.argmax(scores_with_enrolled)])


        far, frr, wmr = [], [], []

        thresholds = sorted(scores)
        labels, scores = np.array(labels), np.array(scores)

        for t in thresholds:
            labels[np.where(scores <= t)[0]] = 'g'
            far.append(false_acceptance_rate(labels, labels_true))
            frr.append(false_rejection_rate(labels, labels_true))
            wmr.append(wrong_member_rate(labels, labels_true))

        fnir, far = np.array(frr) + np.array(wmr), np.array(far)
        idxE = np.nanargmin(np.absolute((far - fnir)))
        eer  = max(far[idxE],fnir[idxE])*100

        eers.append(eer)

    mean_eers.append(np.mean(eers))


In [551]:
#по 1000 семей
for i in range(len(mean_eers)):
    print('nSpeakers =', i + 2, 'EER =', mean_eers[i])

nSpeakers = 2 EER = 0.41777777777777775
nSpeakers = 3 EER = 0.5278260869565218
nSpeakers = 4 EER = 0.7357142857142857
nSpeakers = 5 EER = 0.796
nSpeakers = 6 EER = 0.9146491228070174
nSpeakers = 7 EER = 1.0613953488372092


In [75]:
train = open('./data/train_list.txt', 'r')
count_utterances = 0

for i in train:
    count_utterances += 1
train.close()

In [129]:
import numpy as np

train = open('./data/train_list.txt', 'r')
test = open('./data/test_list.txt', 'a')

i = 0
buffer = ''

for line in train:
    if i % 2:
        id_1, path_1 = buffer.split()
        id_2, path_2 = line.split()
        test.write(str(int(id_1 == id_2)) + ' ' + path_1 + ' ' + path_2 + '\n')
    else:
        buffer = line
    i += 1
    
train.close()
test.close()

In [65]:
#generates train_list [id, path] to each utterance 

path = './data/voxceleb1'
f = open('./data/train_list.txt', 'a')

for id in os.listdir(path):
    for youtube in os.listdir(path + '/' + id):
        for video_id in os.listdir(path + '/' + id + '/' + youtube):
            f.write(id + ' ' + id + '/' + youtube + '/' + video_id + '\n')
f.close()